In [65]:
import os
import re
import json
import io
import requests
import ast
from typing import Optional
from datetime import date
from collections import deque

from dotenv import load_dotenv

from pydantic import BaseModel, Field
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.messages import AIMessage
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_text_splitters import RecursiveCharacterTextSplitter
from unstructured.partition.html import partition_html

In [66]:
# **Approach 2:**

# 1. “Given the following HTML: `html`, write a parser using playwright that pulls out all sections involving clothing”
# 2. Validate and check the generated script for security
# 3. Execute the scripts against the HTML
# 4. “Given the following HTML snippet, extract `ClothingItem` from the data”

In [67]:
load_dotenv()

True

In [68]:
# TODO: Consider an HTML-aware text splitter to not cut off HTML tags
SEARCH_MAX_RESULTS = 2
MAX_SEARCH_DEPTH = 2
CHUNK_SIZE = 20000
MIN_CHUNK_SIZE = 1000
CHUNK_OVERLAP = 100
MAX_RETRIES = 3
MAX_N_LINKS_TO_SEARCH = 3
SEARCH_QUERY = "Ring Jacket coats"

In [69]:
generate_scraping_script_prompt = """
Given the following HTML:
{html}

Write a parser using BeautifulSoup that pulls out all sections involving clothing.
Return the function to extract in a function called `parse_clothing_sections`.
Put all your python code in a code block in markdown format i.e ```code ...```
Return only the python code, nothing else.

The function should look like this:
```python
async def parse_clothing_sections(html: str):
    ...
```

Parsing Script Python:
"""

In [70]:
parse_results_prompt = """
Given the following clothing search results:

Results:
{results}

Extract a list of ClothingItem objects from the results.
Here is the python class definition for ClothingItem:
{clothing_item_class}

Extract only a List[ClothingItem] and nothing else.

clothing_item_list = 
"""

In [71]:
# TODO: Consider fine tuning a smaller model on GPT-4o's output to train it to
# Parse clothing web pages consistently!!!
# That fine tuning would be the kicker feature to show that I really understand ML/AI engineering!!!
llm = ChatOpenAI(
    model="gpt-4o",
    api_key=os.getenv("OPENAI_API_KEY"),
    temperature=0.0,
    streaming=False,
)

fast_llm = ChatOpenAI(
    model="gpt-4o-mini",
    api_key=os.getenv("OPENAI_API_KEY"),
    temperature=0.0,
    streaming=False,
)

In [72]:
class ClothingItem(BaseModel):
    """
    A clothing item extracted from the internet.
    Includes a variety of possible metadata fields.
    """

    id: Optional[str] = Field(
        None, description="Unique identifier for the clothing item"
    )
    name: Optional[str] = Field(
        None, description="Descriptive name of the clothing item"
    )
    brand: Optional[str] = Field(
        None, description="Manufacturer or designer of the clothing item"
    )
    category: Optional[str] = Field(
        None, description="General category of the item, e.g., shirts, pants, dresses"
    )
    subcategory: Optional[str] = Field(
        None,
        description="More specific classification, e.g., t-shirt, jeans, cocktail dress",
    )
    price: Optional[float] = Field(None, description="Current price of the item")
    original_price: Optional[float] = Field(
        None, description="Original price if the item is on sale"
    )
    image_url: Optional[str] = Field(None, description="URL of the item's image")
    color: Optional[str] = Field(None, description="Primary color of the item")
    sizes: Optional[list[str]] = Field(
        None, description="Available sizes or size range"
    )
    material: Optional[str] = Field(
        None, description="Main fabric or material composition"
    )
    gender: Optional[str] = Field(
        None, description="Target gender if applicable (men's, women's, unisex)"
    )
    season: Optional[str] = Field(
        None,
        description="Appropriate season for the item, e.g., summer, winter, all-season",
    )
    style: Optional[str] = Field(
        None, description="Style of the item, e.g., casual, formal, sporty"
    )
    description: Optional[str] = Field(
        None, description="Detailed text description of the item"
    )
    care_instructions: Optional[str] = Field(
        None, description="Instructions for washing and maintaining the item"
    )
    availability: Optional[str] = Field(
        None, description="Availability status, e.g., in stock, out of stock, pre-order"
    )
    average_rating: Optional[float] = Field(
        None, description="Average customer rating of the item"
    )
    num_reviews: Optional[int] = Field(None, description="Number of customer reviews")
    tags: Optional[list[str]] = Field(
        None,
        description="Keywords associated with the item for searching and categorization",
    )
    dimensions: Optional[dict] = Field(
        None, description="Measurements of the item, e.g., length, width, sleeve length"
    )
    weight: Optional[float] = Field(
        None, description="Weight of the item, useful for shipping calculations"
    )
    release_date: Optional[date] = Field(
        None, description="Date when the item was first available or added to inventory"
    )
    sustainability_info: Optional[str] = Field(
        None, description="Information about eco-friendly or ethical production"
    )


class SimpleClothingItem(BaseModel):
    name: str = Field(..., description="Descriptive name of the clothing item")
    price: Optional[float] = Field(None, description="Current price of the item")
    image_url: Optional[str] = Field(None, description="URL of the item's image")
    link: Optional[str] = Field(None, description="URL of the item's page")


class ClothingItemList(BaseModel):
    clothing_items: list[SimpleClothingItem]

In [73]:
def split_html(html: str) -> list[str]:
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=CHUNK_SIZE,
        # min_chunk_size=MIN_CHUNK_SIZE,
        chunk_overlap=CHUNK_OVERLAP,
        length_function=len,
        is_separator_regex=False,
    )
    return text_splitter.split_text(html)

In [74]:
class SearchResult(BaseModel):
    url: str = Field(..., description="URL of the search result")
    html_content: Optional[str] = Field(
        None, description="HTML content of the search result"
    )
    depth: int = Field(0, description="Depth of the search result in the BFS search")


def get_search_results_from_url(url: str, depth: int = 0) -> list[SearchResult]:
    res = []
    response = requests.get(url)
    response.raise_for_status()
    html_content = response.text
    for split_html_content in split_html(html_content):
        res.append(SearchResult(url=url, html_content=split_html_content, depth=depth))
    return res


def get_search_results(search_query: str) -> list[SearchResult]:
    tavily_search = TavilySearchResults(
        max_results=SEARCH_MAX_RESULTS, include_raw_content=True
    )
    search_results = tavily_search.invoke({"query": search_query})
    res = []
    for search_result in search_results:
        print(f"Search result: {search_result}")
        url = search_result["url"]
        try:
            res.extend(get_search_results_from_url(url, 0))
        except Exception as e:
            print(f"Error fetching HTML from {url}: {e}")
            continue
    return res

In [75]:
def get_test_html() -> str:
    with open("test_web_page.html", "r") as f:
        return f.read()

In [76]:
def extract_code_from_markdown(markdown: str) -> str:
    # Try to extract code blocks marked with ```python first
    if "```python" in markdown:
        return markdown.split("```python")[1].split("```")[0]
    # If no python blocks found, try generic code blocks
    elif "```" in markdown:
        return markdown.split("```")[1].split("```")[0]
    return ""

In [77]:
def get_parsing_script(search_results: dict) -> str:
    prompt = PromptTemplate(
        input_variables=["html"],
        template=generate_scraping_script_prompt,
    )
    extract_prompt = prompt.format(html=search_results["content"])
    # print(f"Extract prompt: {extract_prompt}")
    raw_res = AIMessage.model_validate(llm.invoke(extract_prompt)).content
    # print(f"Raw res: {raw_res}")
    return extract_code_from_markdown(raw_res)

In [78]:
def validate_parsing_script(script: str) -> bool:
    """
    Validate the generated parsing script for security concerns.
    Returns True if script passes security checks, False otherwise.
    """
    # List of forbidden operations/keywords that could be dangerous
    forbidden = [
        "import os",
        "import sys",
        "subprocess",
        "eval(",
        "exec(",
        "__import__",
        "open(",
        "write",
        "delete",
        "remove",
    ]

    # Check for forbidden operations
    for item in forbidden:
        if item in script.lower():
            print(f"Security violation: Found forbidden operation '{item}'")
            return False

    # Only allow importing safe parsing libraries
    allowed_imports = {
        "json",
        "asyncio",
        "playwright",
        "beautifulsoup4",
        "bs4",
        "lxml",
        "html.parser",
        "re",
    }

    import_lines = [
        line.strip()
        for line in script.split("\n")
        if line.strip().startswith("import") or line.strip().startswith("from")
    ]

    for line in import_lines:
        module = line.split()[1].split(".")[0]
        if module not in allowed_imports:
            print(f"Security violation: Unauthorized import '{module}'")
            return False

    return True

In [79]:
class FunctionExtractor(ast.NodeVisitor):
    def __init__(self):
        self.functions = []

    def visit_FunctionDef(self, node):
        # Collect regular function name and body
        self.functions.append({
            'name': node.name,
            'type': 'regular',
            'body': ast.unparse(node)
        })
        self.generic_visit(node)

    def visit_AsyncFunctionDef(self, node):
        # Collect async function name and body
        self.functions.append({
            'name': node.name,
            'type': 'async',
            'body': ast.unparse(node)
        })
        self.generic_visit(node)

def extract_functions_from_code(code):
    tree = ast.parse(code)
    extractor = FunctionExtractor()
    extractor.visit(tree)
    return extractor.functions

In [80]:
async def execute_parsing_script(script: str, html: str) -> list:
    local_namespace = {}
    try:
        # Execute the function body in the local namespace
        exec(script, globals(), local_namespace)
        parse_clothing_sections = local_namespace.get("parse_clothing_sections")
        if not parse_clothing_sections:
            raise ValueError("parse_clothing_sections function not found")
        return await parse_clothing_sections(html)
    except Exception as e:
        print(f"Error executing parsing script: {str(e)}")

In [81]:
def result_extractor(raw_results: str) -> ClothingItemList:
    structured_output_llm = llm.with_structured_output(ClothingItemList)
    
    for attempt in range(MAX_RETRIES):
        try:
            raw_res = structured_output_llm.invoke(raw_results)
            print(f"Raw res: {raw_res}")
            return ClothingItemList.model_validate(raw_res)
        except Exception as e:
            if attempt == MAX_RETRIES - 1:  # Last attempt
                raise
            print(f"Attempt {attempt + 1} failed: {str(e)}. Retrying...")

In [82]:
# search_results = split_html(get_test_html())
# print(f"len(search_results): {len(search_results)}")
# print(f"Search results: {search_results[100:110]}")
# print([res_len for res_len in [len(res) for res in search_results]])
# print(
#     f"Average chunk length: {sum(len(chunk) for chunk in search_results) / len(search_results)}"
# )

In [83]:
# TODO: Build a classifer to search results to see if a given result is about a clothing item
# "Is this in the main body of the HTML and is mainly text, or is it just a bunch of numbers?"

In [84]:
html_contains_clothing_info_prompt = """
Given the following HTML snippet:
{html}

Does this HTML contain information about a clothing item or links that are promising for finding clothing items?
Return "true" if it does, "false" otherwise.
"""

In [85]:
get_clothing_item_links_prompt = """
Given the following list of links:
{links}

Extract a list of links that are promising for finding clothing items.
"""

In [86]:
get_n_most_promising_links_prompt = """
Given the following list of links:
{links}

Return the {n} most promising links from the list for finding clothing items.
The link should be something the user clicks on to get to a clothing item page for purchasing.

Links:
"""

In [87]:
def contains_clothing_item_info_or_links(html_chunk: str) -> bool:
    """
    Returns True if the HTML chunk is about a clothing item
    or links that are promising for finding clothing items, False otherwise.
    """
    prompt = PromptTemplate(
        input_variables=["html"],
        template=html_contains_clothing_info_prompt,
    )
    extract_prompt = prompt.format(html=html_chunk)
    raw_res = AIMessage.model_validate(fast_llm.invoke(extract_prompt)).content
    return "true" in raw_res.lower()  # TODO: Consider structured output in the future

In [88]:
def extract_all_links(html_chunk: str) -> list[str]:
    """
    Returns a list of all links in the HTML chunk.
    """
    soup = BeautifulSoup(html_chunk, "html.parser")
    return [link.get("href") for link in soup.find_all("a")]

In [89]:
class ClothingItemLinks(BaseModel):
    links: list[str] = Field(
        ..., description="List of links that are promising for finding clothing items"
    )


def get_clothing_item_links(search_result: SearchResult) -> list[str]:
    """
    Returns a list of links that are promising for finding clothing items.
    """
    raw_links = extract_all_links(search_result.html_content)
    print(f"HTML content: {search_result.html_content}")
    all_links = []
    for link in raw_links:
        if link and not (link.startswith("http://") or link.startswith("https://")):
            all_links.append(search_result.url + link)
        else:
            all_links.append(link)
    print(
        f"All links: {all_links}"
    )  # NOTE: Extracting the links takes ~5 minutes, not a real-time operation
    prompt = PromptTemplate(
        input_variables=["links"],
        template=get_clothing_item_links_prompt,
    )
    get_links_prompt = prompt.format(links=all_links)
    link_extract_llm = llm.with_structured_output(ClothingItemLinks)
    raw_res = link_extract_llm.invoke(get_links_prompt)
    return ClothingItemLinks.model_validate(raw_res).links

In [90]:
# TODO: This is not a real-time feature, architect the frontend to show users search progress while the model is working
# TODO: Why is the search result turning up so much garbage in the HTML?

# TODO: Toplevel pages seem to not have much specific clothing into - will need to dive deeper into websites to get more specific info
# TODO: The scraping logic works, but needs to find web pages where the information actually is - the LLM needs to do some browser control
# Writing a playwright script is good for this!!!
# "Given this following web page, write a playwright script to follow the top N most promising links to get more information about clothing items"

In [91]:
# TODO: Implement BFS with the link traversal script

In [92]:
def get_n_most_promising_links(links: list[str], n: int) -> list[str]:
    """
    Returns the N most promising links from the list.
    """
    prompt = PromptTemplate(
        input_variables=["links", "n"],
        template=get_n_most_promising_links_prompt,
    )
    get_links_prompt = prompt.format(links=links, n=n)
    link_extract_llm = llm.with_structured_output(ClothingItemLinks)
    raw_res = link_extract_llm.invoke(get_links_prompt)
    return ClothingItemLinks.model_validate(raw_res).links
    

In [93]:
async def goto_links(
    raw_html: str, parent_url: str, links: list[str]
) -> list[SearchResult]:
    """
    Finds and clicks the provided links in the raw HTML using Playwright and returns the resulting pages' content.
    """
    results = []

    async with async_playwright() as p:
        browser = await p.chromium.launch()
        page = await browser.new_page()

        # First load the parent page containing the links
        await page.goto(parent_url)
        await page.set_content(raw_html)

        for link in links:
            try:
                # Try to find and click the link in the page
                link_element = (
                    await page.get_by_role("link", exact=True)
                    .filter(has_text=link)
                    .click()
                )

                # Get the content of the new page after clicking
                content = await page.content()
                results.append(SearchResult(url=link, html_content=content))

                # Go back to parent page for next link
                await page.goto(parent_url)
                await page.set_content(raw_html)

            except Exception as e:
                print(f"Error clicking link {link}: {str(e)}")
                continue

        await browser.close()

    return results

In [94]:
async def parse_search_result(search_result: SearchResult) -> list[ClothingItem]:
    print(f"Search chunk: {search_result}")
    try:
        parsing_script = get_parsing_script({"content": search_result})
    except Exception as e:
        print(f"Error getting parsing script: {str(e)}")
        raise e
    if validate_parsing_script(parsing_script):
        # print(f"Raw Parsing script: {parsing_script}")
        extracted_funcs = extract_functions_from_code(parsing_script)
        # print(f"Extracted functions: {extracted_funcs}")
        if len(extracted_funcs) != 1:
            print(f"Expected 1 function, got {len(extracted_funcs)}")
            raise e

        parse_clothing_sections_func = extracted_funcs[0]["body"]
        # print(f"Parsing script:\n{parse_clothing_sections_func}")
        results_safe = validate_parsing_script(parse_clothing_sections_func)
        if not results_safe:
            print("Parse clothing sections function failed security validation")
            raise e

        raw_results = await execute_parsing_script(
            parse_clothing_sections_func, search_result.html_content
        )
        print(f"Results: {raw_results}")
        if raw_results and len(raw_results) > 0:
            try:
                parsed_results = result_extractor(str(raw_results))
                print(f"Parsed results: {parsed_results}")
                return parsed_results.clothing_items
            except Exception as e:
                print(f"Error parsing results: {str(e)}")
                raise e
        else:
            return []
    else:
        print("Generated script failed security validation")
        return []

In [95]:
# TODO: Improve the yield of BFS by classifying pages as "promising" or "not promising"
async def run_clothing_search_crawler(search_query: str) -> list[ClothingItem]:
    """
    Uses BFS and tavily search to find clothing items and crawl search results for
    promising clothing item links.
    """
    queue: deque[SearchResult] = deque()
    search_results = get_search_results(search_query)
    final_result: list[ClothingItem] = []
    # search_results = split_html(get_test_html())
    for search_result in search_results:
        queue.append(search_result)
    while queue:
        search_result = queue.popleft()
        print(f"Parent URL: {search_result.url}")
        if not contains_clothing_item_info_or_links(search_result):
            print(f"Skipping chunk: {search_result}")
            continue

        # Parse the current search result
        parsed_results = await parse_search_result(search_result)
        final_result.extend(parsed_results)

        # Continue BFS by adding the most promising links to the queue
        if search_result.depth < MAX_SEARCH_DEPTH:
            links = get_clothing_item_links(search_result)
            print(f"Links: {links}")
            if not links:
                print(f"No links found for {search_result.url}, continuing search")
                continue
            # TODO: Follow the most promising links or search the current page
            num_links_to_search = min(MAX_N_LINKS_TO_SEARCH, len(links))
            links_to_search = await goto_links(
                search_result.html_content,
                search_result.url,
                get_n_most_promising_links(links, num_links_to_search),
            )
            print(f"Links to search: {links_to_search}")
            for link in links_to_search:
                try:
                    queue.append(
                        get_search_results_from_url(link, search_result.depth + 1)
                    )
                except Exception as e:
                    print(f"Error fetching HTML from {link}: {e}")
                    continue

    return final_result

In [96]:
result = await run_clothing_search_crawler(SEARCH_QUERY)
print(f"Result: {result}")

Search result: {'url': 'https://www.supplyandadvise.com/collections/ring-jacket', 'content': "RING JACKET | Supply & Advise FREE SHIPPING ON ORDERS OVER $250 FREE SHIPPING ON ORDERS OVER $250 CREATE AN ACCOUNT Account Brands ▾ Brands ENGINEERED GARMENTS LEVI'S VINTAGE CLOTHING RANDOLPH ENGINEERING RING JACKET TANNER GOODS Categories ▾ Shoe Care & Accessories Suits & Jackets RING JACKET Established in 1954, Ring Jacket is regarded as one of the finest ready-to-wear producers of suiting and tailored clothing in the world. Ring Jacket works with the finest mills in the world like VBC, Loro Piana & also with smaller local mills to produce proprietary fabrics that have become iconic. RING JACKET Shipping Brands ENGINEERED GARMENTS LEVI'S VINTAGE CLOTHING RANDOLPH ENGINEERING RING JACKET TANNER GOODS Shoe Care & Accessories Suits & Jackets Account"}
Search result: {'url': 'https://www.thearmoury.com/collections/ring-jacket', 'content': 'The Armoury by Ring Jacket Wool/Silk Model 16 Sport Coa

<string>:3: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.


Error executing parsing script: 'NoneType' object has no attribute 'string'
Results: None
HTML content: COAT BLUE","vendor":"RING JACKET","id":"6804965654571","untranslatedTitle":"BALLOON WOOL SPORT COAT BLUE","url":"\/products\/ring-jacket-balloon-wool-sport-coat-blue-1","type":"SUITS \u0026 JACKETS"},"id":"39954994561067","image":{"src":"\/\/www.supplyandadvise.com\/cdn\/shop\/products\/RING-JACKET-BALLOON-WOOL-SPORT-COAT-BLUE-1_86eb841f-6e15-4f53-89a7-ecaed8bdcddb.jpg?v=1648184879"},"sku":"95295425146","title":"46","untranslatedTitle":"46"},{"price":{"amount":737.5,"currencyCode":"USD"},"product":{"title":"BOTTOLI LINEN\/COTTON\/SILK 184 SPORT COAT BEIGE WINDOWPANE","vendor":"RING JACKET","id":"4362088580","untranslatedTitle":"BOTTOLI LINEN\/COTTON\/SILK 184 SPORT COAT BEIGE WINDOWPANE","url":"\/products\/ring-jacket-bottoli-linen-cotton-silk-184-sport-coat-beige-windowpane","type":"SUITS \u0026 JACKETS"},"id":"14586227716","image":{"src":"\/\/www.supplyandadvise.com\/cdn\/shop\/pro

<string>:3: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.


Results: []
HTML content: var meta = {"products":[{"id":6566924025899,"gid":"gid:\/\/shopify\/Product\/6566924025899","vendor":"RING JACKET","type":"SUITS \u0026 JACKETS","variants":[{"id":39334412877867,"price":83700,"name":"HIGH TWIST WOOL\/COTTON GABARDINE SPORT COAT CREAM - 50S","public_title":"50S","sku":"95295573350S"},{"id":39334412910635,"price":83700,"name":"HIGH TWIST WOOL\/COTTON GABARDINE SPORT COAT CREAM - 50","public_title":"50","sku":"95295573350"},{"id":39334412943403,"price":83700,"name":"HIGH TWIST WOOL\/COTTON GABARDINE SPORT COAT CREAM - 52","public_title":"52","sku":"95295573352"}]},{"id":6567010369579,"gid":"gid:\/\/shopify\/Product\/6567010369579","vendor":"RING JACKET","type":"SUITS \u0026 JACKETS","variants":[{"id":39334583271467,"price":105000,"name":"E. THOMAS WOOL\/SILK\/LINEN SPORT COAT BROWN PRINCE OF WALES CHECK - 48","public_title":"48","sku":"95295573548"},{"id":39334583304235,"price":105000,"name":"E. THOMAS WOOL\/SILK\/LINEN SPORT COAT BROWN PRINCE OF

<string>:3: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.


Results: []
HTML content: TWILL FLAT FRONT TROUSER STEEL BLUE - 44","public_title":"44","sku":"95295424744"},{"id":2699790319644,"price":18750,"name":"COTTON TWILL FLAT FRONT TROUSER STEEL BLUE - 46","public_title":"46","sku":"95295424746"},{"id":2699790352412,"price":18750,"name":"COTTON TWILL FLAT FRONT TROUSER STEEL BLUE - 48","public_title":"48","sku":"95295424748"},{"id":2699790385180,"price":18750,"name":"COTTON TWILL FLAT FRONT TROUSER STEEL BLUE - 50","public_title":"50","sku":"95295424750"},{"id":2699790417948,"price":18750,"name":"COTTON TWILL FLAT FRONT TROUSER STEEL BLUE - 52","public_title":"52","sku":"95295424752"},{"id":2699790450716,"price":18750,"name":"COTTON TWILL FLAT FRONT TROUSER STEEL BLUE - 54","public_title":"54","sku":"95295424754"},{"id":2699790483484,"price":18750,"name":"COTTON TWILL FLAT FRONT TROUSER STEEL BLUE - 56","public_title":"56","sku":"95295424756"}]},{"id":10435279247,"gid":"gid:\/\/shopify\/Product\/10435279247","vendor":"RING JACKET","type":"PA

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o in organization org-XwvJU0oTph8nySwcbLMSRZ3R on tokens per min (TPM): Limit 10000, Used 6617, Requested 5147. Please try again in 10.584s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}

In [32]:
# TODO: Consider using partition_html from unstructured to grab the HTML content for each page
# Seems to be more consistent that the custom parsers the AI is writing
# TODO: Compare partition_html vs. writing custom parsers